# Extraction des attestations de mots en 'age' à partir d'un lexique de référence

Trois étapes principales dans le traitement :
  1. extraire d'un corpus (ici frWaC) la liste des attestations d'un procédé morphologique
  2. comparer les items de cette liste avec les entrées d'un lexique de référence (ici lexique.org) et identifier les mots absents du lexique, candidats à la néologie
  3. échantillonnage des résultats

## Extraction des mots en 'age' de frWaC

Le fichier `FRWAC-FULL-utf8.xml` fait 26 Go, c'est très gros. Plutôt que de l'attaquer en Python direct on passe par noSketchEngine.

### NoSketchEngine

[https://www.clarin.si/noske/run.cgi/first_form?corpname=frwac;align=](https://www.clarin.si/noske/run.cgi/first_form?corpname=frwac;align=), sélectionner 'CQL' puis taper la requête `[word=".{3,}ages?"]`  

On recherche sur les formes (`word=`) plutôt que les lemmes sinon on pourrait passer à côté des mots inconnus. `.{3,}` pour au moins 3 caractères, `ages?` pour le suffixe 'age' avec un 's' à la fin ou non.

Une fois qu'on a la concordance on peut avoir la liste des formes dans le menu à gauche 'Frequency > Node forms' puis cliquer sur 'Save' en haut, avec '100000' pour le nombre maximum de lignes et on télécharge les résultats.

On va lire le fichier résultat et stocker les mots dans un dictionnnaire python. Un dictionnaire pour conserver la fréquence de chaque mot. Le fichier a des lignes de commentaires et d'entête dont on ne veut pas, il y aura une petite regex pour ne pas les prendre en compte.

In [1]:
import re
from collections import Counter

# On utilise un objet Counter, un dérivé de dictonnaire plus efficace pour les valeurs numériques
# https://docs.python.org/3.7/library/collections.html#collections.Counter
words_frwac = Counter()

# changer le nom du fichier le cas échéant
with open('frwac_age.txt', encoding='utf-8') as res_frwac:
    for line in res_frwac:
        line = line.rstrip()
        # On cherche un motif regex du type : "pistonnage    9"
        res_search = re.search('([^\t]+)\t([0-9]+)', line)
        # Si la ligne ne contient pas ce motif c'est une ligne d'entête ou de commentaire
        # (Motif = Au moins un charactère qui n'est pas un tab suivi d'un tab suivi d'au moins un chiffre)
        if res_search:
            word, freq = res_search.groups()
            freq = int(freq) # conversion chaîne de car. en entier
            word = word.lower() # passage en minuscule pour supprimer la variation liée à la casse
            if re.search(r"\W|[0-9]|([a-zA-Z])\1{2}|http|Message|Page|Image", word): # r"\w = tout ce qui n'est pas une lettre, ensuite 3x la même lettre
                # On élimine les mots avec certaines caractéristiques qu'on ne veut pas
                # "-" éliminé car préfixation. On veut que le dernier procédé utilisé soit celui en -age.
                # Elimination de 3 lettres à la suite (éliminer une partie des mots mal orthographiés)
                # Eliminer dD et ii ? Pas dangereux ? Message|Page|Image ?
                continue
            # On supprime les s finaux
            if word[-1] == 's':
                word = word[:-1]
            words_frwac[word] += freq

À l'issue de ce traitement on a en mémoire un structure de données avec les mots en 'age' et leurs fréquences.
On peut afficher les 10 mots les plus fréquents par exemple :

In [2]:
for item in words_frwac.most_common(10):
    print(item)

('message', 334281)
('ouvrage', 253193)
('voyage', 217755)
('passage', 183474)
('village', 163300)
('personnage', 158473)
('apprentissage', 141413)
('avantage', 129163)
('langage', 109369)
('paysage', 98767)


Ou simplement voir combien on a de formes différentes

In [3]:
len(words_frwac.keys())

12421

## Comparaison avec lexique.org

#### Stockage en mémoire des mots de lexique.org

Le fichier ici s'appelle `Lexique382.csv` (téléchargé sur la page http://www.lexique.org).

In [4]:
# Un set pour deux raisons : ne garder que les formes uniques, utiliser les fonctions sur les ensembles pour comparer avec les données de corpus
set_words_lexique = set()

# On va aussi garder les mots en 'age' pour voir
words_age_lexique = set()

with open('Lexique382.csv', encoding='utf-8') as lexique:
    for line in lexique:
        if not line.startswith('1_ortho'):
            word = line.split(';')[0]
           # On supprime les s finaux comme pour frwac
            if word[-1] == 's':
                word = word[:-1]
            set_words_lexique.add(word)
            if re.search('.{3,}ages?$', word):
                words_age_lexique.add(word)

In [5]:
print("{} formes différentes (en virant les 's' finaux) dans lexique.org dont {} en age".format(len(set_words_lexique), len(words_age_lexique)))
words_age_lexique

96012 formes différentes (en virant les 's' finaux) dans lexique.org dont 911 en age


{'abatage',
 'abattage',
 'abordage',
 'abreuvage',
 'accastillage',
 'accostage',
 'accrochage',
 'achalandage',
 'adressage',
 'affacturage',
 'affichage',
 'affinage',
 'affouage',
 'affûtage',
 'agnelage',
 'agrafage',
 'agrainage',
 'aguichage',
 'aiguillage',
 'ajustage',
 'ajutage',
 'alliage',
 'allumage',
 'alpage',
 'alunissage',
 'amarrage',
 'ambage',
 'amerrissage',
 'amorçage',
 'ampérage',
 'aménage',
 'ancrage',
 'anthropophage',
 'anti-blindage',
 'anti-braquage',
 'anti-dopage',
 'anti-flingage',
 'anti-mariage',
 'anti-piratage',
 'antiblocage',
 'antidopage',
 'antidémarrage',
 'antiparasitage',
 'apanage',
 'appareillage',
 'appontage',
 'apprentissage',
 'après-rasage',
 'arbitrage',
 'archivage',
 'arpentage',
 'arrachage',
 'arrimage',
 'arrivage',
 'arrosage',
 'arrérage',
 'aréopage',
 'assemblage',
 'assurage',
 'assurance-chômage',
 'asticotage',
 'astiquage',
 'attelage',
 'atterrage',
 'atterrissage',
 'attrapage',
 'autoguidage',
 'avalage',
 'avantage',


#### Comparaison frWac / lexique

On convertit les mots de frWac en ensemble, on applique l'opération de [différence](https://fr.wikipedia.org/wiki/Alg%C3%A8bre_des_parties_d%27un_ensemble#Diff%C3%A9rence)

In [6]:
set_words_frwac = set(words_frwac.keys())
diff = set_words_frwac.difference(set_words_lexique)
print("il y a {} formes en 'age' absentes de lexique.org".format(len(diff)))

il y a 11546 formes en 'age' absentes de lexique.org


On met d'abord le résultat dans deux listes, triées par fréquence décroissante (la liste des mots `new_words` et la liste des fréquences d'occurences `new_words_counts`)

In [7]:
new_words = sorted(diff, key=lambda x: words_frwac[x], reverse=True)
new_words_counts = [words_frwac[w] for w in new_words]

On peut afficher ces formes et leurs fréquences

In [8]:
for word, freq in zip(new_words, new_words_counts):
    print(word, " : ", freq)

paramétrage  :  7558
maillage  :  7264
language  :  6941
elevage  :  6738
dégroupage  :  6564
compostage  :  4745
eclairage  :  4372
lepage  :  3136
carthage  :  2953
chomage  :  2715
menage  :  2478
domage  :  1795
nommage  :  1760
hokage  :  1751
pélerinage  :  1711
calage  :  1695
façonnage  :  1672
eiffage  :  1570
lesage  :  1565
average  :  1522
prototypage  :  1491
depannage  :  1376
recadrage  :  1309
kurokage  :  1308
hermitage  :  1276
mesurage  :  1214
maraîchage  :  1154
fléchage  :  1105
interfaçage  :  1093
relevage  :  1091
temoignage  :  1085
heritage  :  1081
flocage  :  1075
essaimage  :  1059
ferroutage  :  1049
publipostage  :  1013
frontpage  :  1013
affermage  :  991
monnayage  :  957
ensilage  :  953
fraisage  :  950
paillage  :  946
foutage  :  936
chaînage  :  913
delage  :  899
typage  :  875
homepage  :  852
débauchage  :  842
décolletage  :  806
emboutissage  :  804
cmpage  :  803
orpaillage  :  800
lmimage  :  785
equipage  :  762
destockage  :  757
mizukag

caravanage  :  79
entartage  :  79
manpage  :  79
rabattage  :  79
débouillissage  :  79
reperage  :  79
gerbage  :  78
moignage  :  78
échaudage  :  78
déshuilage  :  78
antiremorquage  :  78
ébavurage  :  78
vérouillage  :  77
encapsulage  :  77
agréage  :  77
verouillage  :  77
affutage  :  77
précâblage  :  76
navimage  :  76
catapultage  :  76
barage  :  76
demarage  :  76
potassage  :  76
temoinage  :  75
partimage  :  75
decollage  :  75
redemarrage  :  75
équarissage  :  75
thermolaquage  :  75
pieddepage  :  74
bionettoyage  :  74
maltage  :  74
subpage  :  74
aerogommage  :  74
marchéage  :  74
sexage  :  74
passerillage  :  74
déflocage  :  73
chambrage  :  73
visionage  :  73
infinitestorage  :  73
rattrappage  :  73
survirage  :  72
antidérapage  :  72
linkage  :  72
chainage  :  72
fossoyage  :  72
dessalage  :  72
parrainnage  :  72
racollage  :  72
carriage  :  71
handiplage  :  71
rebasage  :  71
bourrinage  :  71
decoupage  :  71
médicomage  :  71
contrecollage  :  71

légendage  :  25
minimage  :  25
entubage  :  25
aiguilletage  :  25
préformage  :  25
brulage  :  25
gyrobroyage  :  25
dégrippage  :  24
encourrage  :  24
dévernissage  :  24
décalottage  :  24
kinescopage  :  24
desimlockage  :  24
mélangeage  :  24
débalage  :  24
bottage  :  24
rentrage  :  24
prémontage  :  24
préremplissage  :  24
tracage  :  24
enradiage  :  24
croutage  :  24
conteneurimage  :  24
grimpage  :  24
cyclonage  :  24
repartage  :  24
frisage  :  24
délettrage  :  24
déroctage  :  24
aquaplanage  :  24
decapsulage  :  24
dinklage  :  24
marrage  :  24
networkvantage  :  24
flockage  :  24
unepage  :  24
dragonsage  :  24
embrochage  :  24
animage  :  24
loupage  :  23
plaxage  :  23
talutage  :  23
refendage  :  23
telepeage  :  23
avivage  :  23
depèage  :  23
debridage  :  23
dépolissage  :  23
remoulage  :  23
ratrappage  :  23
microphage  :  23
prélaquage  :  23
hackage  :  23
grundlage  :  23
détatouage  :  23
amerissage  :  23
décortiquage  :  23
actimage  : 

fêtage  :  12
líusage  :  12
traversage  :  12
statophage  :  12
decoffrage  :  12
drageonnage  :  12
girage  :  12
aprendissage  :  12
dlzwencodepage  :  12
grandparentage  :  12
enfouissage  :  12
fouttage  :  12
gratinage  :  12
gimage  :  12
antage  :  12
fouinage  :  12
caryotypage  :  12
leurrage  :  12
resucrage  :  12
cherchage  :  12
bibusage  :  12
bracage  :  12
lusage  :  12
bellimage  :  12
tournassage  :  12
draînage  :  12
croquage  :  12
pupitrage  :  12
crochage  :  12
billonnage  :  12
wallimage  :  12
gardinnage  :  12
vêtage  :  12
aysage  :  12
allogarage  :  12
linchage  :  12
kitage  :  12
paramêtrage  :  12
nextpage  :  12
pivotage  :  12
comatage  :  12
switchage  :  11
farfouillage  :  11
cabanage  :  11
décalquage  :  11
discutage  :  11
spammage  :  11
ecoemballage  :  11
bouffage  :  11
fliquage  :  11
ichtyophage  :  11
haubannage  :  11
défléchage  :  11
touchage  :  11
prenage  :  11
remérage  :  11
ddamage  :  11
saproxylophage  :  11
foulonnage  :  11


autrepage  :  7
déplissage  :  7
jointoyage  :  7
recasage  :  7
forrage  :  7
désenrobage  :  7
pouillage  :  7
strage  :  7
auvetage  :  7
amménage  :  7
breakage  :  7
dõimage  :  7
pucage  :  7
lerinage  :  7
déhoussage  :  7
decrassage  :  7
desménage  :  7
covage  :  7
désamarrage  :  7
automaquillage  :  7
montrage  :  7
driveimage  :  7
garrottage  :  7
monopage  :  7
architecturage  :  7
recylcage  :  7
betonnage  :  7
sculptage  :  7
resquillage  :  7
graphiquage  :  7
prülage  :  7
pressvoyage  :  7
sérogroupage  :  7
mdedressage  :  7
détrancanage  :  7
baissage  :  7
rembarrage  :  7
détritiphage  :  7
taquage  :  7
prérepérage  :  7
boucanage  :  7
kelstage  :  7
ardage  :  7
cinémontage  :  7
étallonage  :  7
surpéage  :  7
revissage  :  7
manuportage  :  7
jumellage  :  7
mélage  :  7
visitage  :  7
atterisage  :  7
pitonnage  :  7
ébousage  :  7
psychosondage  :  7
lézardage  :  7
calinage  :  7
turage  :  7
pelanage  :  7
renomage  :  7
dedommage  :  7
cadmiage  :  7


...

gossage  :  1
unsinage  :  1
disoulage  :  1
geoforage  :  1
parkingsauvage  :  1
arètage  :  1
kilmètrage  :  1
pèresonnage  :  1
epurage  :  1
mayticvillage  :  1
nukuninkage  :  1
sangage  :  1
apprentiissage  :  1
affichaage  :  1
flcage  :  1
mythophage  :  1
tomatage  :  1
leemage  :  1
quotapage  :  1
klaxonnage  :  1
fifivoyage  :  1
palissonage  :  1
interbloquage  :  1
néocartilage  :  1
engaage  :  1
decriptage  :  1
lnpinglage  :  1
décommage  :  1
surkiffage  :  1
multipexage  :  1
rollcage  :  1
couraillage  :  1
easystage  :  1
pénétrage  :  1
lesportugaislabrennesauvage  :  1
pmessage  :  1
galerage  :  1
degivreage  :  1
dépsitage  :  1
depannarage  :  1
monophasage  :  1
pélenirage  :  1
pourentage  :  1
réalèsage  :  1
témoiganage  :  1
ejarrage  :  1
homonage  :  1
déboxage  :  1
moillage  :  1
duparamétrage  :  1
helitreuillage  :  1
sâengage  :  1
revignettage  :  1
hurlage  :  1
oloriage  :  1
flippage  :  1
blériotplage  :  1
vernage  :  1
égaillage  :  1
déquic

kraftage  :  1
avanntage  :  1
scrotophage  :  1
feillage  :  1
reliftage  :  1
autobronzage  :  1
microfimage  :  1
teubitage  :  1
préchauaffage  :  1
rebêchage  :  1
stpckage  :  1
monsevrage  :  1
reagreage  :  1
autoéquilibrage  :  1
sakage  :  1
ecovilage  :  1
marmamassage  :  1
chicotage  :  1
microblindage  :  1
cblage  :  1
suremenage  :  1
sanage  :  1
vientrage  :  1
intimesage  :  1
chôniage  :  1
bissage  :  1
trucquage  :  1
publiphage  :  1
insiderpage  :  1
bizzutage  :  1
numerimage  :  1
bâfrage  :  1
enthomophage  :  1
micromassage  :  1
hertitage  :  1
thermosiphonnage  :  1
ivisage  :  1
larmage  :  1
semoussage  :  1
mutiplexage  :  1
amalgamage  :  1
sogimage  :  1
embellissage  :  1
parétage  :  1
manchplage  :  1
enjonquage  :  1
zarcophage  :  1
apprenbtissage  :  1
dâassemblage  :  1
désargentage  :  1
cronometrage  :  1
decalagescompactage  :  1
mantage  :  1
epissurage  :  1
youvoxvoyage  :  1
polishcage  :  1
nabaztage  :  1
laictage  :  1
calcifrage  :  

candissage  :  1
imbrage  :  1
librage  :  1
vellage  :  1
tripatouilage  :  1
fredonnage  :  1
addictionspage  :  1
jasage  :  1
douillage  :  1
goujonnage  :  1
srremballage  :  1
darosage  :  1
pailletage  :  1
déliérage  :  1
prérincage  :  1
patinnage  :  1
inmage  :  1
glougloutage  :  1
éfficage  :  1
vidangeage  :  1
incroyage  :  1
désentralaçage  :  1
decoolage  :  1
discoursmasquage  :  1
dégrage  :  1
épincetage  :  1
statusage  :  1
recimentage  :  1
seriage  :  1
cromage  :  1
conscientisage  :  1
débasculage  :  1
saussage  :  1
cheminimage  :  1
anrtiblocage  :  1
embrossage  :  1
díespionnage  :  1
winnage  :  1
multinage  :  1
numériscopage  :  1
tatanage  :  1
débourgeonage  :  1
craspouillage  :  1
decordage  :  1
festtage  :  1
enchassage  :  1
fagocitage  :  1
déralinguage  :  1
zozotage  :  1
etincelage  :  1
quayage  :  1
imzage  :  1
fornage  :  1
ossage  :  1
kauage  :  1
etoupage  :  1
dossierspartage  :  1
surmodelage  :  1
fourchetage  :  1
dégrassage  :  1

pourléchage  :  1
dépensage  :  1
distribcodage  :  1
découlage  :  1
bucherônnage  :  1
testboximage  :  1
pétrophage  :  1
relpokage  :  1
montpetitvillage  :  1
ectopage  :  1
remarginage  :  1
dingbatpage  :  1
ciopage  :  1
renettoyage  :  1
dumouillage  :  1
suicidage  :  1
microsevrage  :  1
socotecavantage  :  1
ablage  :  1
danpalmage  :  1
décyptage  :  1
lafilledurivage  :  1
déssappage  :  1
industriespionage  :  1
typesdosage  :  1
retortage  :  1
éhritage  :  1
callipage  :  1
ensemblage  :  1
copurage  :  1
artibrage  :  1
norimage  :  1
gaussage  :  1
visaage  :  1
polyusage  :  1
emboutiellage  :  1
nicoulascage  :  1
desreportage  :  1
rougage  :  1
limonage  :  1
critiquage  :  1
relançage  :  1
volstage  :  1
postimage  :  1
feulletage  :  1
magicomage  :  1
écadrage  :  1
xandercage  :  1
démaraboutage  :  1
ccouplage  :  1
entreillage  :  1
nofilesyspage  :  1
bosage  :  1
maillochage  :  1
auguillage  :  1
désappareillage  :  1
floccage  :  1
sonystage  :  1
cend

updatepage  :  1
cléscourtage  :  1
dépistatage  :  1
korobarrage  :  1
fistockage  :  1
biturage  :  1
malimage  :  1
tandage  :  1
caboage  :  1
bioguidage  :  1
plastronnage  :  1
grinchissage  :  1
crypatage  :  1
rebroussage  :  1
eclissage  :  1
brutage  :  1
butonage  :  1
englobage  :  1
langugage  :  1
maîtimage  :  1
sérotriage  :  1
skuattage  :  1
hiretage  :  1
bringandage  :  1
dupaysage  :  1
quotientage  :  1
doutage  :  1
affachage  :  1
replanissage  :  1
renvidage  :  1
réamménage  :  1
skiage  :  1
clientaffacturage  :  1
tagspage  :  1
comtétemoignage  :  1
ensuquage  :  1
déquipage  :  1
multireportage  :  1
bafouiilage  :  1
éclaiçrage  :  1
alsacerecyclage  :  1
retalcage  :  1
ahbillage  :  1
blaquotage  :  1
assosage  :  1
monéage  :  1
coinsclassepage  :  1
iximage  :  1
degauchissage  :  1
aprilage  :  1
santépage  :  1
ducerage  :  1
parkeergarage  :  1
autogonflage  :  1
pareage  :  1
sclavage  :  1
jonctionnage  :  1
burelage  :  1
loadandcallimage  :  1


On écrit ces résultats dans un fichier, on trie les mots par fréquence décroissante

In [9]:
with open('diff-age-frwac-lexique.txt', 'w') as f_diff:
    for word, freq in zip(new_words, new_words_counts):
        print(word, freq, file=f_diff)

## Comparaison avec le lefff

#### Stockage en mémoire des mots du lefff

Le fichier ici s'appelle `lefff-3.4.elex` (téléchargé sur la page https://www.labri.fr/perso/clement/lefff/)

In [10]:
# Un set pour deux raisons : ne garder que les formes uniques, utiliser les fonctions sur les ensembles pour comparer avec les données de corpus
set_words_lefff = set()

# On va aussi garder les mots en 'age' pour voir
words_age_lefff = set()

with open('lefff-3.4.elex') as lexique:
    for line in lexique:
        word = line.split('\t')[0]
        # On supprime les s finaux comme pour frwac
        if word[-1] == 's':
            word = word[:-1]
        set_words_lefff.add(word)
        if re.search('.{3,}ages?', word):
            words_age_lefff.add(word)

In [11]:
print("{} formes différentes dans le lefff (en virant les 's' finaux) dont {} en age".format(len(set_words_lefff), len(words_age_lefff)))

350813 formes différentes dans le lefff (en virant les 's' finaux) dont 4611 en age


#### Comparaison

In [12]:
diff_lefff = set_words_frwac.difference(set_words_lefff)
print("il y a {} formes en 'age' absentes de lefff".format(len(diff_lefff)))

il y a 9981 formes en 'age' absentes de lefff


Sortie dans un fichier, on trie les mots par fréquence décroissante

In [13]:
with open('diff-age-frwac-lefff.txt', 'w') as f_diff:
    for item in sorted(diff_lefff, key=lambda x: words_frwac[x], reverse=True):
        print(item, words_frwac[item], file=f_diff)

## Echantillonnage

On veut séparer la liste des nouveaux mots en `k` sous-listes. S'il y a `N` nouveaux mots on cherche donc `q` et `r` entier tel que `N == q*k + r`.

In [14]:
k = 20
N = len(new_words)
q = N // k  # Division entière i.e. division arrondie vers zéro
r = N % k   # Modulo i.e. donne le reste de la division entière

On crée une liste de liste pour contenir les sous-listes.

In [15]:
sublists = [new_words[i*q:(i+1)*q] for i in range(k)]  # Réparti les q*k premiers mots en k sous-listes
sublists[-1] += new_words[-r:]  # Ajoute le reste à la dernière sous-liste

On tire désormais 100 mots dans chaque sous-liste et on les écris dans un fichier.

**Note :** On "seed" le générateur de nombre aléatoire pour avoir toujours le même résultat. Utiliser `seed(m)` avec `m` différent changera le résultat, si nécessaire.

In [18]:
from random import seed, sample
seed(0)

with open('sampled_words.txt', 'w') as file:
    for sublist in sublists:
        sampled = sample(sublist, 100)
        for word in sorted(sampled, key=lambda w: -words_frwac[w]):
            print(word, " ", words_frwac[word], file=file)